In [3]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_selection import RFE, RFECV
import Feature_Importance_Funs


diabetes = pd.read_csv("diabetes.csv")
diabetes.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,1.0.2,2.0.1,3.0.1,4.0.1,5.0.1,6.0.1,7.0,8.0,feature_3,healthy_food
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0,0,0,0,5.0,1.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,1,0,0,0,0,0,0,0,0.0,0.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,1,5.0,1.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,0,0,0,0,0,1,0,0,2.0,2.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,0,0,0,1,0,0,0,0,2.0,2.0


In [2]:
diabetes.shape

(253680, 44)

In [7]:
X = diabetes.drop(['Diabetes_012'], axis = 1)
Y = diabetes['Diabetes_012']


nsim = 2
cv = 5
test = Feature_Importance_Funs.RF_Feature_Importance(X, Y, nsim, cv)

01

In [9]:
feature_importance = pd.DataFrame({'Features': X.columns, 'Importance': test.apply(np.mean, axis = 0)})
feature_importance['Importance'] = round(feature_importance['Importance'], 4) * 100
feature_importance = feature_importance.sort_values(by = ['Importance'], ascending = False).reset_index(drop = True)
feature_importance.head(10)

,Features,Importance
0,feature_3,17.69
1,feature_1,16.07
2,GenHlth,9.04
3,HighBP,8.57
4,1.0,8.15
5,0.0,8.10
6,feature_2,6.73
7,BMI_log,4.92
8,BMI,4.74
9,DiffWalk,4.48


In [11]:
# Defining input and target variables
X = diabetes.drop(['Diabetes_012'], axis = 1)
Y = diabetes['Diabetes_012']

# Splitting the data into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

# Running RFE with random forest
auto_feature_selection = RFECV(estimator = RandomForestClassifier(n_estimators = 100, max_depth = 5, min_samples_split = 10, min_samples_leaf = 7), step = 1, min_features_to_select = 2, cv = 5).fit(X_train, Y_train)

# Extracting features that got slected
print(X_train.columns[auto_feature_selection.support_])

Index(['HighBP', 'HighChol', 'BMI', 'GenHlth', 'Age', 'BMI_log', 'feature_1',
       '0.0', '1.0', 'feature_3'],
      dtype='object')
